# Interview Challenge - Relax

The data is available as two attached CSV files: takehome_user_engagement.csv, takehome_users.csv

**1]** A user table ("takehome_users") with data on 12,000 users who signed up for the product in the last two years. 

This table includes:

name: the user's name

object_id: the user's id

email: email address

creation_source: how their account was created. This takes on one
of 5 values:
○ PERSONAL_PROJECTS: invited to join another user's personal workspace
○ GUEST_INVITE: invited to an organization as a guest (limited permissions)
○ ORG_INVITE: invited to an organization (as a full member)
○ SIGNUP: signed up via the website
○ SIGNUP_GOOGLE_AUTH: signed up using Google

Authentication (using a Google email account for their login
id)

creation_time: when they created their account

last_session_creation_time: unix timestamp of last login

opted_in_to_mailing_list: whether they have opted into receiving
marketing emails

enabled_for_marketing_drip: whether they are on the regular
marketing email drip

org_id: the organization (group of users) they belong to

invited_by_user_id: which user invited them to join (if applicable).

**2]** A usage summary table ("takehome_user_engagement") that has a row for each day that a user logged into the product.


**Goal**

Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven­day period, identify which factors predict future user adoption.

In [1692]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import datetime

In [1693]:
cwd = os.getcwd()

In [1694]:
# import csvs
users = pd.read_csv(f"{cwd}/data/takehome_users.csv", encoding='latin-1')
user_engagement = pd.read_csv(f"{cwd}/data/takehome_user_engagement.csv")

In [1695]:
users.head(5)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [1696]:
# explore and clean data

# null last_session_creation_time likely means still active? or was never active after creation?
# check rows where creation_time and last_session_creation_time are same

# are non-null invited_by_user_id more likely to remain?
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [1697]:
# use fromtimestamp() to convert unix time 'last_session_creation_time' to datetime object 
# can't convert null to int for fromtimestamp()
last_session_not_null = users['last_session_creation_time'].dropna()

#print(last_session_not_null)
users['new_last_session_creation_time'] = last_session_not_null.apply(lambda t: datetime.datetime.fromtimestamp(t))

In [1698]:
users.isna().sum()

object_id                            0
creation_time                        0
name                                 0
email                                0
creation_source                      0
last_session_creation_time        3177
opted_in_to_mailing_list             0
enabled_for_marketing_drip           0
org_id                               0
invited_by_user_id                5583
new_last_session_creation_time    3177
dtype: int64

Explore user engagement df

In [1699]:
user_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [1700]:
user_engagement.isna().sum()

time_stamp    0
user_id       0
visited       0
dtype: int64

In [1701]:
user_engagement.head(5)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


# Find adopted users

*Users who have logged in 3 separate DAYS in one 7 day period*

In [1702]:
user_engagement['time_stamp'] = pd.to_datetime(user_engagement['time_stamp'])

In [1703]:
user_engagement['date'] = user_engagement['time_stamp'].dt.date

In [1704]:
user_engagement['user_id'].nunique()

8823

In [1705]:
user_engagement.head(5)

,time_stamp,user_id,visited,date
0,2014-04-22 03:53:30,1,1,2014-04-22
1,2013-11-15 03:45:04,2,1,2013-11-15
2,2013-11-29 03:45:04,2,1,2013-11-29
3,2013-12-09 03:45:04,2,1,2013-12-09
4,2013-12-25 03:45:04,2,1,2013-12-25


In [1706]:
# group together user_id with 3 or more different dates

In [1707]:
# user_engagement has 8823 unique user_id, so null last_session_creation_time likely means not active user?
d = user_engagement['user_id'].value_counts()

In [1708]:
d = pd.DataFrame(d, index=None)
active_3_days = d[d['user_id'] >= 3]
active_3_days = active_3_days.rename(columns={'user_id':'visits'})
active_3_days.reset_index(inplace=True)
active_3_days.head(5)


,index,visits
0,3623,606
1,906,600
2,1811,593
3,7590,590
4,8068,585


In [1709]:
selected = user_engagement.loc[user_engagement['user_id'].isin(active_3_days['index'])]

In [1710]:
user_engagement['user_id'].nunique()

8823

In [1711]:
selected['user_id'].nunique()

2248

In [1712]:
selected['time_stamp'] = selected['time_stamp'].dt.floor('d').astype(np.int64)
selected = selected.sort_values(['user_id', 'time_stamp']).drop_duplicates()

# need at least 3 login days, so window 3
grouped = selected.groupby('user_id')['time_stamp'].rolling(window=3)
time_delta = pd.to_timedelta((grouped.max()-grouped.min())).dt.days

/var/folders/v_/wdytmqyx1377p1m8yrz0tdrr0000gn/T/ipykernel_11625/2938769241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['time_stamp'] = selected['time_stamp'].dt.floor('d').astype(np.int64)


In [1713]:
# max - min time_stamp for each user_id, NaN if doesn't meet criteria
time_delta

user_id        
2        1          NaN
         2          NaN
         3         24.0
         4         26.0
         5         22.0
                   ... 
11991    207905     NaN
         207906     NaN
         207907     8.0
         207908    22.0
         207909    32.0
Name: time_stamp, Length: 201002, dtype: float64

In [1714]:
# user_id meeting grouped requirement over 7 days
adopted_user_ids = time_delta[time_delta == 7].index.get_level_values('user_id').tolist()
unique_adopted_users = time_delta[time_delta == 7].index.get_level_values('user_id').drop_duplicates().tolist()

In [1715]:
# number of unique adopted users
len(unique_adopted_users)

1358

In [1716]:
unique_adopted_users

[2,
 10,
 33,
 42,
 50,
 60,
 63,
 69,
 74,
 81,
 82,
 87,
 133,
 135,
 146,
 153,
 160,
 165,
 168,
 185,
 188,
 197,
 200,
 202,
 203,
 209,
 214,
 230,
 245,
 247,
 263,
 265,
 275,
 280,
 297,
 298,
 305,
 310,
 311,
 322,
 341,
 345,
 351,
 362,
 363,
 370,
 401,
 418,
 430,
 445,
 450,
 460,
 462,
 469,
 471,
 478,
 483,
 492,
 494,
 497,
 502,
 509,
 510,
 512,
 518,
 520,
 522,
 529,
 535,
 540,
 553,
 564,
 572,
 601,
 603,
 605,
 618,
 628,
 632,
 634,
 639,
 669,
 679,
 680,
 717,
 724,
 728,
 754,
 772,
 783,
 786,
 804,
 845,
 851,
 869,
 874,
 882,
 885,
 901,
 906,
 907,
 912,
 928,
 932,
 934,
 937,
 943,
 980,
 985,
 1009,
 1013,
 1017,
 1018,
 1027,
 1039,
 1055,
 1061,
 1072,
 1089,
 1093,
 1094,
 1099,
 1106,
 1107,
 1119,
 1128,
 1129,
 1136,
 1145,
 1150,
 1151,
 1155,
 1156,
 1163,
 1173,
 1186,
 1196,
 1202,
 1212,
 1235,
 1245,
 1250,
 1274,
 1280,
 1290,
 1318,
 1319,
 1320,
 1327,
 1339,
 1343,
 1345,
 1350,
 1357,
 1368,
 1396,
 1407,
 1410,
 1411,
 1421,
 1

1358 are adopted users of the 2248 that had >= 3 visits on at least 3 days and 8823 with non-null 'last_session_created_time'

# Identify factors that predict future user adoption

Feature engineering

In [1717]:
users.head(5)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,new_last_session_creation_time
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,2014-04-21 23:53:30
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2014-03-30 23:45:04
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,2013-03-19 19:14:52
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,2013-05-22 04:09:28
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,2013-01-22 05:14:20


In [1718]:
len(users)

12000

In [1719]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   object_id                       12000 non-null  int64         
 1   creation_time                   12000 non-null  object        
 2   name                            12000 non-null  object        
 3   email                           12000 non-null  object        
 4   creation_source                 12000 non-null  object        
 5   last_session_creation_time      8823 non-null   float64       
 6   opted_in_to_mailing_list        12000 non-null  int64         
 7   enabled_for_marketing_drip      12000 non-null  int64         
 8   org_id                          12000 non-null  int64         
 9   invited_by_user_id              6417 non-null   float64       
 10  new_last_session_creation_time  8823 non-null   datetime64[ns]
dtypes:

In [1720]:
# 'object_id' in users is 'user_id' in user_engagement
#users.rename(columns={'object_id':'user_id'}, inplace=True)

In [1721]:
users['creation_time'] = pd.to_datetime(users['creation_time'])
# users['invited_by_user_id'].astype(np.int64, errors='ignore')

In [1722]:
# account creation to last login time
users['creation_delta'] = users['new_last_session_creation_time'] - users['creation_time']

In [1723]:
adopted_users = pd.DataFrame(adopted_user_ids, columns=['user_id'])

In [1724]:
adopted_users['adopted_user'] = 1
adopted_users

,user_id,adopted_user
0,2,1
1,10,1
2,10,1
3,10,1
4,10,1
...,...,...
6672,11975,1
6673,11975,1
6674,11975,1
6675,11975,1


In [1725]:
all_users = pd.merge(users, adopted_users, how='outer',
                  left_on='object_id', right_on='user_id')

In [1726]:
all_users = pd.get_dummies(all_users, columns=['creation_source'])

In [1727]:
all_users.head(5)

,object_id,creation_time,name,email,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,new_last_session_creation_time,creation_delta,user_id,adopted_user,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,1.398139e+09,1,0,11,10803.0,2014-04-21 23:53:30,-1 days +20:00:00,NaN,NaN,1,0,0,0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,1.396238e+09,0,0,1,316.0,2014-03-30 23:45:04,135 days 20:00:00,2.0,1.0,0,1,0,0,0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,1.363735e+09,0,0,94,1525.0,2013-03-19 19:14:52,-1 days +20:00:00,NaN,NaN,0,1,0,0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,1.369210e+09,0,0,1,5151.0,2013-05-22 04:09:28,0 days 20:00:00,NaN,NaN,1,0,0,0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,1.358850e+09,0,0,193,5240.0,2013-01-22 05:14:20,4 days 19:00:00,NaN,NaN,1,0,0,0,0


In [1728]:
all_users['adopted_user'] = all_users['adopted_user'].fillna(0)

In [1729]:
all_users.isna().sum()

object_id                                 0
creation_time                             0
name                                      0
email                                     0
last_session_creation_time             3177
opted_in_to_mailing_list                  0
enabled_for_marketing_drip                0
org_id                                    0
invited_by_user_id                     7890
new_last_session_creation_time         3177
creation_delta                         3177
user_id                               10642
adopted_user                              0
creation_source_GUEST_INVITE              0
creation_source_ORG_INVITE                0
creation_source_PERSONAL_PROJECTS         0
creation_source_SIGNUP                    0
creation_source_SIGNUP_GOOGLE_AUTH        0
dtype: int64

In [1730]:
# to look at only adopted users, 'object_id' = 'user_id'
adopted_users_df = all_users.loc[all_users['object_id'].isin(adopted_user_ids)]

In [1731]:
adopted_users_df.head(5)

,object_id,creation_time,name,email,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,new_last_session_creation_time,creation_delta,user_id,adopted_user,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,1.396238e+09,0,0,1,316.0,2014-03-30 23:45:04,135 days 20:00:00,2.0,1.0,0,1,0,0,0
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,1.401833e+09,1,1,318,4143.0,2014-06-03 18:08:03,502 days 20:00:00,10.0,1.0,0,1,0,0,0
10,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,1.401833e+09,1,1,318,4143.0,2014-06-03 18:08:03,502 days 20:00:00,10.0,1.0,0,1,0,0,0
11,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,1.401833e+09,1,1,318,4143.0,2014-06-03 18:08:03,502 days 20:00:00,10.0,1.0,0,1,0,0,0
12,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,1.401833e+09,1,1,318,4143.0,2014-06-03 18:08:03,502 days 20:00:00,10.0,1.0,0,1,0,0,0


In [1732]:
len(adopted_users_df)

6677

In [1733]:
adopted_users_df.isna().sum()

object_id                                0
creation_time                            0
name                                     0
email                                    0
last_session_creation_time               0
opted_in_to_mailing_list                 0
enabled_for_marketing_drip               0
org_id                                   0
invited_by_user_id                    2890
new_last_session_creation_time           0
creation_delta                           0
user_id                                  0
adopted_user                             0
creation_source_GUEST_INVITE             0
creation_source_ORG_INVITE               0
creation_source_PERSONAL_PROJECTS        0
creation_source_SIGNUP                   0
creation_source_SIGNUP_GOOGLE_AUTH       0
dtype: int64

In [ ]:
# were they invited by a user? -- creation_source -- is that user id also very active user id or kept user id 

In [ ]:
# model feature importance